# Has COVID-19 impacted reviews of scented candles negatively?

First, we'll add some libraries to our classpath.

In [1]:
%%classpath add mvn
tech.tablesaw tablesaw-beakerx 0.38.1
tech.tablesaw tablesaw-excel 0.38.1
tech.tablesaw tablesaw-aggregate 0.38.1

And add some associated imports.

In [2]:
%import tech.tablesaw.api.*
%import tech.tablesaw.io.xlsx.XlsxReader

%import java.time.LocalDate
%import java.time.LocalDateTime
%import java.util.function.Function

%import static java.lang.Math.sqrt
%import static java.time.Month.JANUARY
%import static tech.tablesaw.aggregate.AggregateFunctions.count
%import static tech.tablesaw.aggregate.AggregateFunctions.countTrue
%import static tech.tablesaw.aggregate.AggregateFunctions.mean
%import static tech.tablesaw.api.QuerySupport.and
%import static tech.tablesaw.io.xlsx.XlsxReadOptions.builder

Plus a helper method since the two graphs are very similar.

In [3]:
tableFor = { String url ->
    def table = new XlsxReader().read(builder(new URL(url)).build())
    table.addColumns(
        DateColumn.create('YearMonth', table.column('Date').collect { LocalDate.of(it.year, it.month, 15) })
    )
    table
}

plots = { table, Color lineColor, Color markerColor ->
    def janFirst2017 = LocalDate.of(2017, JANUARY, 1)
    Function from2017 = { r -> r.dateTimeColumn('Date').isAfter(janFirst2017) }
    Function top3 = { r -> r.intColumn('CandleID').isLessThanOrEqualTo(3) }

    def byMonth = table.sortAscendingOn('Date')
            .where(and(from2017, top3))
            .summarize('Rating', mean).by('YearMonth')
    def byDate = table.sortAscendingOn('Date')
            .where(and(from2017, top3))
            .summarize('Rating', mean).by('Date')

    def averaged = new Line(x: byMonth.dateColumn('YearMonth').toList(), y: byMonth.nCol('Mean [Rating]').toList(), width: 4, color: lineColor)
    def scatter = new Points(x: byDate.dateTimeColumn('Date').toList(), y: byDate.nCol('Mean [Rating]').toList(), color: markerColor)
    [averaged, scatter]
}
OutputCell.HIDDEN

Let's create a line representing when COVID was first reported.

In [4]:
def covidReported = LocalDate.of(2020, JANUARY, 20)
line = new ConstantLine(x: [covidReported], color: Color.RED, style: StrokeType.DASH, showLabel: true)
OutputCell.HIDDEN

Now the graph for scented candles:

In [5]:
def scentedUrl = 'https://github.com/paulk-asert/groovy-data-science/blob/master/subprojects/Candles/src/main/resources/Scented_all.xlsx?raw=true'
scented = tableFor(scentedUrl)
def (sAverage, sScatter) = plots(scented, Color.BLUE, new Color(190, 120, 64, 80))
plot = new TimePlot(title: "Top 3 scented candles Amazon reviews 2017-2020", xLabel: 'Date', yLabel: 'Average daily rating (1-5)')
plot << sAverage
plot << sScatter
plot << line

Now the graph for unscented candles:

In [6]:
def unscentedUrl = 'https://github.com/paulk-asert/groovy-data-science/blob/master/subprojects/Candles/src/main/resources/Unscented_all.xlsx?raw=true'
unscented = tableFor(unscentedUrl)
def (uAverage, uScatter) = plots(unscented, Color.MAGENTA, new Color(32, 190, 64, 80))
plot = new TimePlot(title: "Top 3 unscented candles Amazon reviews 2017-2020", xLabel: 'Date', yLabel: 'Average daily rating (1-5)')
plot << uAverage
plot << uScatter
plot << line

In [7]:
def start2020 = LocalDateTime.of(2020, JANUARY, 1, 0, 0)
Function from2020 = { r -> r.dateTimeColumn('Date').isAfter(start2020) }

def candidates = ['[Nn]o scent', '[Nn]o smell', '[Dd]oes not smell like', "[Dd]oesn't smell like", "[Cc]an't smell",
                  '[Cc]annot smell', '[Ff]aint smell', '[Ff]aint scent', "[Dd]on't smell", '[Ll]ike nothing']
int[] months = scented.column('Date').collect { it.month.value - 1 }
def reviewFlags = scented.column('Review').collect { review -> candidates.any { review =~ it } }
scented.addColumns(IntColumn.create('Month', months), BooleanColumn.create('Noscent', reviewFlags))

def byMonth2020 = scented.where(from2020).sortAscendingOn('Date')
        .summarize('Noscent', countTrue, count).by('Month')
def indices = 0..<byMonth2020.size()
double[] nsprop = indices.collect { byMonth2020[it].with { it.getDouble('Number True [Noscent]') / it.getDouble('Count [Noscent]') } }
double[] se = indices.collect { sqrt(nsprop[it] * (1 - nsprop[it]) / byMonth2020[it].getDouble('Count [Noscent]')) }
double[] barLower = indices.collect { nsprop[it] - se[it] }
double[] barHigher = indices.collect { nsprop[it] + se[it] }
byMonth2020.addColumns(DoubleColumn.create('nsprop', nsprop), DoubleColumn.create('barLower', barLower), DoubleColumn.create('barHigher', barHigher))
def title = 'Proportion of top 5 scented candles on Amazon mentioning lack of scent by month 2020'
def labels = 'JFMAMJJASON'.toList()
plot = new Plot(title: title, xLabel: 'Month', yLabel: 'Proportion of reviews', xTickLabelsVisible: false)
labels.indices.each{ plot << new Text(x: it, y:0.005, text: labels[it], showPointer: false, color: Color.WHITE) }
plot << new Bars(x: byMonth2020.nCol('Month').toList(), y: byMonth2020.nCol('nsprop').toList(), width: 0.8)
plot << new Stems(y: byMonth2020.nCol('barLower').toList(), base: byMonth2020.nCol('nsprop').toList())
plot << new Stems(y: byMonth2020.nCol('barHigher').toList(), base: byMonth2020.nCol('nsprop').toList())